In [2]:
# importing all necessary libraries to run the code
import re,string
import numpy as np
import pandas as pd
import keras_metrics
import tensorflow.keras
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation,Embedding
# using the variable sw to hold all stopwords that are in English
sw = stopwords.words('english')

Using TensorFlow backend.


In [3]:
#reading dataset
data=pd.read_csv('Womens Clothing E-Commerce Reviews.csv')
#data=pd.read_csv('train.csv',encoding = 'ISO-8859-1')

In [4]:
data['Class Name'].value_counts()

Dresses           6319
Knits             4843
Blouses           3097
Sweaters          1428
Pants             1388
Jeans             1147
Fine gauge        1100
Skirts             945
Jackets            704
Lounge             691
Swim               350
Outerwear          328
Shorts             317
Sleep              228
Legwear            165
Intimates          154
Layering           146
Trend              119
Casual bottoms       2
Chemises             1
Name: Class Name, dtype: int64

In [5]:
data.dropna(inplace=True)


In [6]:
data.reset_index(inplace=True)
print(data.isnull().sum())
data.head(2)

index                      0
Unnamed: 0                 0
Clothing ID                0
Age                        0
Title                      0
Review Text                0
Rating                     0
Recommended IND            0
Positive Feedback Count    0
Division Name              0
Department Name            0
Class Name                 0
dtype: int64


,index,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
1,3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants


In [7]:
#applying pre-processing steps to remove stopwords and words of size less than 2
data['Review Text'] = data['Review Text'].apply(lambda x: x.split())
wordsEng = stopwords.words('english')
data['Review Text'] = data['Review Text'].apply(lambda x:[item for item in x if item not in wordsEng])
data['Review Text'] = data['Review Text'].apply(lambda x: [w for w in x if len(w)>2])
data['Review Text'] = data['Review Text'].apply(lambda x: " ".join(x))
data.head(2)

,index,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,2,2,1077,60,Some major design flaws,high hopes dress really wanted work me. initia...,3,0,0,General,Dresses,Dresses
1,3,3,1049,50,My favorite buy!,"love, love, love jumpsuit. fun, flirty, fabulo...",5,1,0,General Petite,Bottoms,Pants


In [8]:
#removing airline company names as pre-processing
testList=[]
def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

def strip_all_entities(text):
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)


for t in range(len(data)):
    testList.append(strip_all_entities(strip_links(data['Review Text'][t])))

In [9]:
#converting testList[] list into dataframe for further processing
dat = pd.DataFrame(np.array(testList))
dat.head(5)

,0
0,high hopes dress really wanted work me initial...
1,love love love jumpsuit fun flirty fabulous ev...
2,This shirt flattering due adjustable front tie...
3,love tracy reese dresses one petite feet tall ...
4,aded basket hte last mintue see would look lik...


In [10]:
# applying pre-processing to remove special symbols, numbers and converting into lower case
sw = stopwords.words('english')
twitterSentiment=[]
for i in range(len(dat)):
    review = re.sub('[^a-zA-Z]', ' ',dat[0][i])
    review = re.sub('[/(){}\[\]\|@!,;]', ' ',dat[0][i])
    review = re.sub('[^0-9a-zA-Z #+_♥️]', ' ',dat[0][i])#Remove bad symbols
    
    review = re.sub(r'\d+', '',review)
    review = review.lower()
    review = review.split()

    review = [token for token in review if token not in sw]
    review=' '.join(review)
    review=' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",review).split())
    twitterSentiment.append(review)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
count_vectorizer = TfidfVectorizer(ngram_range=(1,1))
X_train = count_vectorizer.fit_transform(X_train)
X_test = count_vectorizer.transform(X_test)

In [11]:
from sklearn.model_selection import train_test_split
#from sklearn import preprocessing
#lab_enc = preprocessing.LabelEncoder()
#lab_enc.fit(data['gender'])
#data['gender']=lab_enc.transform(data['gender'])
X_train, X_test, Y_train, Y_test = train_test_split(dat[0],data.Rating, test_size = 0.3, random_state = 42)

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report
lr=LogisticRegression()
sgd=lr.fit(X_train, Y_train).predict(X_test)
print(accuracy_score(Y_test,sgd))
print(classification_report(Y_test,sgd))

C:\Users\Muhammad Umer\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Muhammad Umer\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6290896762163078
              precision    recall  f1-score   support

           1       0.80      0.04      0.07       219
           2       0.32      0.07      0.11       456
           3       0.36      0.38      0.37       674
           4       0.43      0.26      0.32      1256
           5       0.71      0.94      0.81      3294

    accuracy                           0.63      5899
   macro avg       0.52      0.34      0.34      5899
weighted avg       0.59      0.63      0.58      5899



In [18]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,classification_report
print("SVC")
svm = SVC(kernel='linear', C=2.0, random_state=500)
predictionSVM=svm.fit(X_train, Y_train).predict(X_test)
print(accuracy_score(Y_test,predictionSVM))
print(classification_report(Y_test,predictionSVM))

SVC
0.6202746228174267
              precision    recall  f1-score   support

           1       0.45      0.24      0.31       219
           2       0.30      0.23      0.26       456
           3       0.35      0.42      0.38       674
           4       0.41      0.31      0.35      1256
           5       0.77      0.86      0.81      3294

    accuracy                           0.62      5899
   macro avg       0.46      0.41      0.42      5899
weighted avg       0.60      0.62      0.60      5899



In [15]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score,classification_report
clf = SGDClassifier(max_iter=1100, tol=1e-3)
calibrated_clf = CalibratedClassifierCV(clf, cv=5, method='isotonic')
pred=calibrated_clf.fit(X_train, Y_train).predict(X_test)
print(accuracy_score(Y_test,pred))
print(classification_report(Y_test,pred))

0.6287506357009662
              precision    recall  f1-score   support

           1       0.44      0.16      0.23       219
           2       0.28      0.08      0.12       456
           3       0.36      0.42      0.39       674
           4       0.40      0.24      0.30      1256
           5       0.74      0.93      0.82      3294

    accuracy                           0.63      5899
   macro avg       0.44      0.37      0.37      5899
weighted avg       0.58      0.63      0.58      5899



In [16]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100, random_state=50) 
predictionRF=rfc.fit(X_train, Y_train).predict(X_test)
print(accuracy_score(Y_test,predictionRF))
print(classification_report(Y_test,predictionRF))

0.5855229699949144
              precision    recall  f1-score   support

           1       0.00      0.00      0.00       219
           2       0.40      0.01      0.03       456
           3       0.38      0.15      0.22       674
           4       0.34      0.06      0.10      1256
           5       0.61      0.99      0.75      3294

    accuracy                           0.59      5899
   macro avg       0.35      0.24      0.22      5899
weighted avg       0.49      0.59      0.47      5899



In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import SGDClassifier
clf2 = SGDClassifier(max_iter=1100, tol=1e-3)
clf1 = LogisticRegression()
eclf1 = VotingClassifier(estimators=[
        ('lr', clf1), ('sgd', clf2)],voting='hard')
predictionVC=eclf1.fit(X_train, Y_train).predict(X_test)
print(accuracy_score(Y_test,predictionVC))
print(classification_report(Y_test,predictionVC))

C:\Users\Muhammad Umer\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Muhammad Umer\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6316324800813697
              precision    recall  f1-score   support

           1       0.37      0.15      0.21       219
           2       0.28      0.12      0.16       456
           3       0.37      0.41      0.39       674
           4       0.44      0.23      0.30      1256
           5       0.73      0.93      0.82      3294

    accuracy                           0.63      5899
   macro avg       0.44      0.37      0.38      5899
weighted avg       0.58      0.63      0.59      5899



In [10]:
#converting twitterSentiment[] list into dataframe for serving it to keras tokenizer
dataSetFinal = pd.DataFrame(np.array(twitterSentiment))
dataSetFinal.head(2)

,0
0,high hopes dress really wanted work initially ...
1,love love love jumpsuit fun flirty fabulous ev...


In [37]:
tokenizer = tensorflow.keras.preprocessing.text.Tokenizer(num_words=5000, lower=True,split=' ',filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(dataSetFinal[0].values)
#print(tokenizer.word_index)  # To see the dicstionary
X = tokenizer.texts_to_sequences(dataSetFinal[0].values)
X = tensorflow.keras.preprocessing.sequence.pad_sequences(X)

In [22]:
from keras import optimizers
#Deep Learning Network Structure
model_conv = Sequential()
model_conv.add(Embedding(2500,100, input_length=X.shape[1]))
model_conv.add(Dropout(0.5))
model_conv.add(Conv1D(64, 5, activation='relu'))
model_conv.add(MaxPooling1D(pool_size=4))
model_conv.add(LSTM(100))
model_conv.add(Dense(5, activation='softmax'))
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_conv.compile(loss='binary_crossentropy', optimizer='sgd',metrics=['accuracy','mae',keras_metrics.precision(), keras_metrics.recall()])

In [43]:
from keras import optimizers
#Deep Learning Network Structure
model_conv = Sequential()
model_conv.add(Embedding(5000,100, input_length=X.shape[1]))
model_conv.add(Dropout(0.5))
model_conv.add(LSTM(100))
model_conv.add(Dense(5, activation='softmax'))
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_conv.compile(loss='categorical_crossentropy', optimizer='sgd',metrics=['accuracy','mae',keras_metrics.precision(), keras_metrics.recall()])

In [ ]:
batch_size=64
Y = pd.get_dummies(data['Rating']).values
X_train, X_valid, Y_train, Y_valid = train_test_split(X,Y, test_size = 0.30)
#Here we train the Network.
pred=model_conv.fit(X_train, Y_train, batch_size =batch_size, epochs =10, verbose =2,validation_data=(X_valid,Y_valid))
pred

Train on 13763 samples, validate on 5899 samples
Epoch 1/10


In [44]:
#for  LSTM only
batch_size=128
Y = pd.get_dummies(data['Rating']).values
X_train, X_valid, Y_train, Y_valid = train_test_split(X,Y, test_size = 0.30)
#Here we train the Network.
pred=model_conv.fit(X_train, Y_train, batch_size =batch_size, epochs =25, verbose =2,validation_data=(X_valid,Y_valid))
pred

Train on 13763 samples, validate on 5899 samples
Epoch 1/25
 - 28s - loss: 1.1358 - acc: 0.5595 - mean_absolute_error: 0.2360 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.9453 - val_acc: 0.5984 - val_mean_absolute_error: 0.1930 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/25
 - 22s - loss: 0.8759 - acc: 0.6313 - mean_absolute_error: 0.1901 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.8966 - val_acc: 0.6143 - val_mean_absolute_error: 0.1985 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/25
 - 22s - loss: 0.7929 - acc: 0.6701 - mean_absolute_error: 0.1754 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.8840 - val_acc: 0.6371 - val_mean_absolute_error: 0.1805 - val_precision: 0.4615 - val_recall: 0.0316
Epoch 4/25
 - 22s - loss: 0.7410 - acc: 0.6906 - mean_absolute_error: 0.1652 - precision: 0.5870 - recall: 0.0539 - val_loss: 0.9114 - val_acc: 0.6311 - val_mean_absolute_error: 0.1781 - val_precision: 0.2857 - val_recall

In [45]:
#LSTM Only
score=[]
score=model_conv.evaluate(X_valid,Y_valid,verbose=2,batch_size=batch_size)
#keras.metrics.binary_accuracy(Y_valid,pred)
print("score: %.2f" %(score[0]))
print("validation accuracy: %.2f" % (score[1]))
print("recall: %.2f" %(score[4]))
print("Precision: %.2f" % (score[3]))

score: 1.74
validation accuracy: 0.58
recall: 0.22
Precision: 0.27


In [18]:
#LSTM
score=[]
score=model_conv.evaluate(X_valid,Y_valid,verbose=2,batch_size=batch_size)
#keras.metrics.binary_accuracy(Y_valid,pred)
print("score: %.2f" %(score[0]))
print("validation accuracy: %.2f" % (score[1]))
print("recall: %.2f" %(score[4]))
print("Precision: %.2f" % (score[3]))

score: 0.40
validation accuracy: 0.82
recall: 0.00
Precision: 0.00


In [14]:
from gensim.models import Word2Vec
from sklearn import preprocessing
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data['Rating'])
data = data
tokenized_data = data['Review Text'].apply(lambda x: x.split())
model_w2v = Word2Vec( tokenized_data, size=200, window=5, min_count=2,sg = 1, hs = 0, negative = 10, workers= 2, seed = 34)
model_w2v.train(tokenized_data, total_examples= len(data['Review Text']), epochs=20)
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += model_w2v[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
        return vec
wordvec_arrays = np.zeros((len(tokenized_data), 200))
for i in range(len(tokenized_data)):
    wordvec_arrays[i,:] = word_vector(tokenized_data[i], 200)
data_feature_3 = pd.DataFrame(wordvec_arrays)
df_tex = data_feature_3
df_cat = data['Rating']
x_train, x_test, y_train, y_test = train_test_split(df_tex, df_cat, test_size=0.3, random_state=2, stratify=df_cat)
y_train = y_train.astype('int')


C:\Users\Muhammad Umer\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Muhammad Umer\Anaconda3\lib\site-packages\ipykernel\__main__.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report
lr=LogisticRegression()
sgdW2V=lr.fit(x_train, y_train).predict(x_test)
print(accuracy_score(y_test,sgdW2V))
print(classification_report(y_test,sgdW2V))

C:\Users\Muhammad Umer\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Muhammad Umer\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6314629598236989
              precision    recall  f1-score   support

           1       0.50      0.15      0.24       207
           2       0.30      0.09      0.14       408
           3       0.41      0.37      0.39       739
           4       0.41      0.25      0.31      1287
           5       0.72      0.94      0.81      3258

    accuracy                           0.63      5899
   macro avg       0.47      0.36      0.38      5899
weighted avg       0.58      0.63      0.58      5899



In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report
rfc = RandomForestClassifier(n_estimators=100, random_state=52,max_depth=10) 
predRfc=rfc.fit(x_train, y_train).predict(x_test)
print(accuracy_score(y_test,predRfc))
print(classification_report(y_test,predRfc))

0.5867096117986099
              precision    recall  f1-score   support

           1       0.00      0.00      0.00       207
           2       0.67      0.00      0.01       408
           3       0.43      0.21      0.28       739
           4       0.31      0.08      0.12      1287
           5       0.61      0.98      0.76      3258

    accuracy                           0.59      5899
   macro avg       0.40      0.25      0.23      5899
weighted avg       0.51      0.59      0.48      5899



C:\Users\Muhammad Umer\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import SGDClassifier
from sklearn import linear_model
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
clf = linear_model.SGDClassifier(max_iter=1100, tol=1e-3)
calibrated_clf = CalibratedClassifierCV(clf, cv=5, method='isotonic')
pred=calibrated_clf.fit(x_train, y_train).predict(x_test)
print(accuracy_score(y_test,pred))
print(classification_report(y_test,pred))

0.63383624343109
              precision    recall  f1-score   support

           1       0.41      0.19      0.26       207
           2       0.29      0.10      0.14       408
           3       0.40      0.44      0.42       739
           4       0.41      0.22      0.29      1287
           5       0.73      0.93      0.82      3258

    accuracy                           0.63      5899
   macro avg       0.45      0.38      0.39      5899
weighted avg       0.58      0.63      0.59      5899



In [10]:
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

clf = SGDClassifier(loss='log')
lr=LogisticRegression()
vc=VotingClassifier(estimators=[('lr', lr), ('clf', clf),],voting='hard')
sgd=vc.fit(x_train, y_train).predict(x_test)
print(accuracy_score(y_test,sgd))
print(classification_report(y_test,sgd))

C:\Users\Muhammad Umer\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Muhammad Umer\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6316324800813697
              precision    recall  f1-score   support

           1       0.43      0.12      0.18       207
           2       0.32      0.13      0.18       408
           3       0.43      0.42      0.43       739
           4       0.44      0.20      0.27      1287
           5       0.71      0.95      0.81      3258

    accuracy                           0.63      5899
   macro avg       0.46      0.36      0.37      5899
weighted avg       0.58      0.63      0.58      5899



In [11]:
svm = SVC(kernel='linear', C=2.0, random_state=2)
svmPred=svm.fit(x_train, y_train).predict(x_test)
print(accuracy_score(y_test,svmPred))
print(classification_report(y_test,svmPred))

0.6314629598236989
              precision    recall  f1-score   support

           1       0.38      0.12      0.18       207
           2       0.31      0.05      0.09       408
           3       0.38      0.54      0.44       739
           4       0.42      0.19      0.27      1287
           5       0.74      0.93      0.82      3258

    accuracy                           0.63      5899
   macro avg       0.45      0.37      0.36      5899
weighted avg       0.58      0.63      0.58      5899

